In [50]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [51]:
og_survey_df = pd.read_csv('comma-survey.csv', index_col='RespondentID')
new_survey_df = pd.read_csv('new_comma_survey.csv', index_col='RespondentID')
survey_df = pd.concat([og_survey_df, new_survey_df])

In [52]:
question_cols = list(survey_df.columns)[:7]
demographic_cols = list(survey_df.columns)[7:11]

In [53]:
question_cols

['In your opinion, which sentence is more gramatically correct?',
 'Prior to reading about it above, had you heard of the serial (or Oxford) comma?',
 'How much, if at all, do you care about the use (or lack thereof) of the serial (or Oxford) comma in grammar?',
 'How would you write the following sentence?',
 'When faced with using the word "data", have you ever spent time considering if the word was a singular or plural noun?',
 'How much, if at all, do you care about the debate over the use of the word "data" as a singluar or plural noun?',
 'In your opinion, how important or unimportant is proper use of grammar?']

In [55]:
# Encode demographic responses
demographics = survey_df[demographic_cols].to_numpy()
# Define the ordinal features pipeline
enc = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder()),
    ('scaler', StandardScaler())
])
enc.fit(demographics)
X_demographics = enc.transform(demographics)

# Fit models
lr = LogisticRegression(multi_class='multinomial')
models = []
for q in question_cols:
    model = lr.fit(X_demographics, survey_df[q].to_numpy())
    models.append(model)

ValueError: Input contains NaN

In [33]:
models

dict_keys(['categories', 'dtype', 'handle_unknown', 'unknown_value'])